# Checking enviroment states and action sets

In [ ]:
import sys
import os.path
sys.path.append("..")
from SAC_Agent_tf import *
from ENV_DETAILS import *
import os
#os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64:/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64:/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64"

from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\BipedalWalker\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/BipedalWalker/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_sac/' # Linux

ENV = "Ant-v2"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [ ]:

env = gym.make(ENV)
env

In [ ]:

max_steps = env._max_episode_steps
max_steps

In [ ]:
s = env.observation_space.sample()
s

In [ ]:
env.observation_space

In [ ]:
env.observation_space.shape

In [ ]:
s = env.reset()
s[0]

In [ ]:
a = env.action_space.sample()
a, a.shape, a.reshape((1, len(a))).shape

In [ ]:
env.step(a)

In [ ]:
env.action_space.sample()

### Training/Hyperparam run

In [ ]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "logs_hyper/fit_sac/"


In [ ]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_sac"
    project_name = "keras_tunning_soft_tf"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/sac_tf/",
                        end_of_episode = EPISODES, evaluation_epoch = 4000, training_steps = 700000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,        
                discount = 0.99,
                #discount_min = 0.98, discount_max = 0.99,      
                      
                min_lr_actor_critic = 0.00001, max_lr_actor_critic= 0.001,
                #lr_actor_min = 0.0001, lr_actor_max = 0.001,
                #lr_critic_1_min = 0.0001, lr_critic_1_max = 0.001, 
                
                dense_unit=256,
                #dense_min = 128, dense_max = 256,
                
                environment_name=ENV,
                #tau = 0.005,
                tau_min = 0.001, tau_max = 0.1,
                
                num_layers_act = 2, num_layers_crit =2,
                #max_num_layers_act = 2, max_num_layers_crit=2
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 20,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")
        writer= "logs_hyper/fit_sac_tf/"

        training_steps = 1000000
        learning_rate= 0.001
        entropy_factor = 0.1
        discount = 0.99
        dense_units_actor = [128,64]
        num_layers_actor = 2
        dense_units_critic = [128,64]
        num_layers_crit = 2
        time_to_update= 100
        end_of_episode = 300
        tau = 0.01
        
        run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, num_layers_actor, num_layers_crit, writer, end_of_episode,
                      environment_name = ENV,reward_scaler = 10, return_agent = False,lr_actor= 0.001, lr_critic_1= 0.001, lr_alpha = 0.001, tau = 0.001)
        

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=5):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:

val_env = gym.make(ENV)
dir = r"Hyperparam_kt_sac"

for trials in tuner.oracle.get_best_trials(num_trials=4):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
    
#env_model = tuner.get_best_models()[0]
best_hps = tuner.get_best_hyperparameters(4)[-1]  # Access best hyperparameter configuration
print(f"Best Hyperparameters: {best_hps}")

env_model = tuner.hypermodel.build(best_hps)  # Build the model with best hyperparameters
env_model = rerun_training(training_steps  = 2000000, 
                           save_factor = 50000,  
                           model_path = './checkpoints/SACagent',
                           sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, 
                           sucess_criteria_value = SUCESS_CRITERIA_VALUE, 
                           return_agent = True, 
                           model = env_model
                           )

final_rewards = final_evaluation(env_model, val_env, n_tries=200, exploration=exploration_tech, sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH,  video_name = "./sac_"+exploration_tech+"_video.mp4")
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
training_steps = 2000000
discount  = best_hp["discount"]
lr_actor  = best_hp["lr_actor"]
lr_critic_1  = best_hp["lr_critic_1"]
tau  = best_hp["tau"]

end_of_episode = EPISODES
sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH
sucess_criteria_value = SUCESS_CRITERIA_VALUE
save_factor=50000
return_agent = True
reward_scaler = 100
environment_name=ENV

n_dense_layers_actor = trials.hyperparameters.values["n_dense_layers_actor"]
dense_layers_actor = []
for i in range(n_dense_layers_actor):
    dense_layers_actor.append(trials.hyperparameters.values['dense_units_act_'+str(i)])

n_dense_layers_critic = trials.hyperparameters.values["n_dense_layers_critic"]
dense_layers_critic = []
for i in range(n_dense_layers_critic):
    dense_layers_critic.append(trials.hyperparameters.values['dense_units_crit_'+str(i)])

env_model = run_training(training_steps = training_steps,
                         discount = discount,  
                         dense_units_act = dense_layers_actor,  
                         dense_units_crit = dense_layers_critic, 
                         num_layer_a = n_dense_layers_actor, 
                         num_layer_c = n_dense_layers_critic, 
                         writer = writer, 
                         end_of_episode = end_of_episode, 
                         save_factor = save_factor, 
                         sucess_criteria_epochs = sucess_criteria_epochs , 
                         sucess_criteria_value = sucess_criteria_value, 
                         environment_name = environment_name, 
                         reward_scaler = reward_scaler , 
                         return_agent = return_agent,
                         lr_actor = lr_actor, 
                         lr_critic_1 = lr_critic_1, 
                         tau = tau )

In [ ]:
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./sac_"+exploration_tech+"_video.mp4")
